# UPGMA Algorithm

1. Assign each taxon to its own cluster

2. Define one leaf for each taxon; place it at height 0

3. While more than two clusters:

    - Determine two clusters $i$ and $j$ with smallest $d_{ij}$
    
    - Define a new cluster $C_k = C_i ∪ C_j$

    - Define a node $k$ with children $i$ and $j$, place it at height $d_{ij}/2$
    
    - Replace clusters $i$ and $j$ with $k$
    
    - Compute distance between $k$ and other clusters
    
4. Join last two clusters $i$ and $j$ by root at height $d_{ij}/2$

In [1]:
import numpy as np

In [27]:
# Input matrices must be square and symmetric with -1s on the diagonal

example = [
    [-1, 19, 27, 8, 33, 18, 13],
    [19, -1, 31, 18, 36, 1, 13],
    [27, 31, -1, 26, 41, 32, 29],
    [8, 18, 26, -1, 31, 17, 14],
    [33, 36, 41, 31, -1, 35, 28],
    [18, 1, 32, 17, 35, -1, 12],
    [13, 13, 29, 14, 28, 12, -1]
]

In [66]:
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g']

In [ ]:
# example = {
#     0: [],
#     1: [19],
#     2: [27, 31],
#     3: [8, 18, 26],
#     4: [33, 36, 41, 31],
#     5: [18, 1, 32, 17, 35],
#     6: [13, 13, 29, 14, 28, 12]
# }

In [90]:
def upgma_iteration(matrix, original, replaced_indices, codes):
    """
    matrix = square matrix input
    original = the first matrix input
    """
    original = np.array(matrix, dtype=float)

    # Convert matrix input to numpy array
    matrix = np.array(matrix, dtype=float)
    dim = matrix.shape

    # Check to make sure matrix size is valid for the algorithm
    assert(dim[0] == dim[1]), "Input is not a square matrix."
    assert(dim[0 != 1]), "Everything is grouped."

    # Find the next shortest pairwise distance and the corresponding indices (groups) in the CURRENT matrix
    current_min = np.max(matrix)
    pair = (-1,-1)
    for r in range(dim[0]):
        for c in range(dim[1]):
            if r <= c: #stay out of upper traingular matix
                continue
            elif current_min > matrix[r,c]:
                current_min = matrix[r,c]
                pair = (r, c) #pair are the indices in the context of the current matrix

    print(pair)
    
    #Let's keep track of the groupings
    new_codes = [None] * (len(codes) - 1)
    i = 0
    while i < len(new_codes):
        if i == min(pair):
            new_codes[i] = codes[i] + codes[max(pair)]
            i = i + 1
        elif i >= max(pair):
            new_codes[i] = codes[i+1]
            i = i+1
        else:
            new_codes[i] = codes[i]
            i = i+1

    # Keep track of the number of individuals grouped together that you are about to join
    paired_count1 = len(codes[pair[0]])
    paired_count2 = len(codes[pair[1]])
    
    # Calculate the depth of the new branch
    depth = current_min / 2

    #Take slices
    slice1 = matrix[:, pair[0]]
    slice1[pair[1]] = -1
    slice2 = matrix[:, pair[1]]
    slice2[pair[0]] = -1

    print("paired_count1", paired_count1)
    print("paired_count2", paired_count2)
    # Calculate the mean pairwise distances with other sequences in new matrix.
    pairwise_mean = sum([slice1*paired_count1, slice2*paired_count2]) / (paired_count1+paired_count2)
    

    # Remove the second occurence of -1
    second_negative_index = np.where(pairwise_mean==-1)[0][1]
    pairwise_mean = np.delete(pairwise_mean, second_negative_index)

    # Construct the new matrix
    index_del = max(pair)
    index_replace = min(pair)
    new_matrix = np.delete(matrix, index_del, axis=0)
    new_matrix = np.delete(new_matrix, index_del, axis=1)

    for i in range(len(pairwise_mean)):
        new_matrix[index_replace,i] = pairwise_mean[i]
        new_matrix[i,index_replace] = pairwise_mean[i]

    new_matrix[new_matrix == 0] = -1
    
    return {"new_matrix":new_matrix, "depth":depth, "original_matrix": original, "index_replace":index_replace, "new_codes": new_codes}

In [91]:
def upgma_full(matrix, codes):

    original = np.array(matrix, dtype=float)
    current_matrix = np.array(matrix, dtype=float)
    dim = current_matrix.shape[0]
    replaced_indices = []
    groupings_stack = []
    
    while dim > 1:
        if dim <= 3:
            print(current_matrix)
        iterresults = upgma_iteration(current_matrix, original, replaced_indices, codes)
        replaced_indices.append(iterresults["index_replace"])
        current_matrix = iterresults["new_matrix"]
        groupings_stack.append(codes)
        print(codes)
        codes = iterresults["new_codes"]
        dim = current_matrix.shape[0]


In [92]:
stack = upgma_full(example, letters)

(5, 1)
paired_count1 1
paired_count2 1
['a', 'b', 'c', 'd', 'e', 'f', 'g']
(3, 0)
paired_count1 1
paired_count2 1
['a', 'bf', 'c', 'd', 'e', 'g']
(4, 1)
paired_count1 1
paired_count2 2
['ad', 'bf', 'c', 'e', 'g']
(1, 0)
paired_count1 3
paired_count2 2
['ad', 'bfg', 'c', 'e']
[[-1.  29.  32.6]
 [29.  -1.  41. ]
 [32.6 41.  -1. ]]
(1, 0)
paired_count1 1
paired_count2 5
['adbfg', 'c', 'e']
[[-1. 34.]
 [34. -1.]]
(-1, -1)
paired_count1 1
paired_count2 1


IndexError: index 1 is out of bounds for axis 0 with size 1

In [54]:
#Math for computing the final matrix value ADBFGC --> E. It should be 34 = (32.6*5 + 41*1) / 6

x_list = [1, 2, 3, 4, 5, 6, 7, 8, 9 ,10]
y_list = [1, 2, 3, 4, 5, 6, 7, 8, 9 ,10]
z_list = [1, 2, 3, 4, 5, 6, 7, 8, 9 ,10]
for x in x_list:
    for y in y_list:
        for z in z_list:
            hello = (32.6*x+41*y) / z
            if hello == 34:
                print(x, y, z)


5 1 6
